In [ ]:
import piplite
import micropip
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
await piplite.install(['pandas'])
await piplite.install(['matplotlib'])
await piplite.install(['scipy'])
await piplite.install(['seaborn'])
await micropip.install(['ipywidgets'],keep_going=True)
await micropip.install(['tqdm'],keep_going=True)

In [ ]:
#This function will download the dataset into your browser 

from pyodide.http import pyfetch

async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())

In [ ]:
path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Data%20files/module_5_auto.csv'
#you will need to download the dataset; if you are running locally, please comment out the following 
await download(path, "auto.csv")
path="auto.csv"
df = pd.read_csv(path)
df.to_csv('module_5_auto.csv')

In [ ]:
df=df._get_numeric_data()
df.head()

In [ ]:
#Definimos una funcion de ploteo
def DistributionPlot(RedFunction, BlueFunction, RedName, BlueName, Title):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    ax1 = sns.distplot(RedFunction, hist=False, color="r", label=RedName)
    ax2 = sns.distplot(BlueFunction, hist=False, color="b", label=BlueName, ax=ax1)

    plt.title(Title)
    plt.xlabel('Price (in dollars)')
    plt.ylabel('Proportion of Cars')

    plt.show()
    plt.close()

In [ ]:
#Definimos otra funcion de ploteo
def PollyPlot(xtrain, xtest, y_train, y_test, lr,poly_transform):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))
    
    
    #training data 
    #testing data 
    # lr:  linear regression object 
    #poly_transform:  polynomial transformation object 
 
    xmax=max([xtrain.values.max(), xtest.values.max()])

    xmin=min([xtrain.values.min(), xtest.values.min()])

    x=np.arange(xmin, xmax, 0.1)


    plt.plot(xtrain, y_train, 'ro', label='Training Data')
    plt.plot(xtest, y_test, 'go', label='Test Data')
    plt.plot(x, lr.predict(poly_transform.fit_transform(x.reshape(-1, 1))), label='Predicted Function')
    plt.ylim([-10000, 60000])
    plt.ylabel('Price')
    plt.legend()

In [ ]:
#Creamos el valor y, el cual es el valor objetivo
y_data = df['price']
#Definimos el valor x, el cual es todo el data set excepto la columna y
x_data=df.drop('price',axis=1)

In [ ]:
#Hacemos una division aleatoria de las particiones de datos de entrenamienot y prueba
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.10, random_state=1)

print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])


In [ ]:
# Creamos una nueva particion de entreamineto como parte de la preguna 1
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_data, y_data, test_size=0.4, random_state=0) 
print("number of test samples :", x_test1.shape[0])
print("number of training samples:",x_train1.shape[0])

In [ ]:
#Libreria de regresion lineal
from sklearn.linear_model import LinearRegression

In [ ]:
#Creamos un objeto que contendra la regresion lineal
lre=LinearRegression()

In [ ]:
#Alimentamos la regreion lineal con la columna deseada en x y nuestra y
lre.fit(x_train[['horsepower']], y_train)

In [ ]:
#Calculamos la r cuadrada en el dato de prueba
lre.score(x_test[['horsepower']], y_test)

In [ ]:
#Ahora comparamos el r cuadrada de los datos de entrenamiento
lre.score(x_train[['horsepower']], y_train)

In [ ]:
#Hacemos un ejercicio de r cuadrada con un set de prueba del 40% como parte del ejercicio 2
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_data, y_data, test_size=0.4, random_state=0)
lre.fit(x_train1[['horsepower']],y_train1)
lre.score(x_test1[['horsepower']],y_test1)

In [ ]:
#Libreria para el cross validation score
from sklearn.model_selection import cross_val_score

In [ ]:
#Creamos una cariable
Rcross = cross_val_score(lre, x_data[['horsepower']], y_data, cv=4)
#El parametro cv determina el numero de fold, para este caso usamos 4
#Imprimimos el ccontenido
Rcross

In [ ]:
#Calculamos el promedio y la desviacion estandar de nuestra estimacion
print("The mean of the folds are", Rcross.mean(), "and the standard deviation is" , Rcross.std())"

In [ ]:
#Podemos usar el error cuadratico negativo omo una metrica, utilzadndo la metricad
-1 * cross_val_score(lre,x_data[['horsepower']], y_data,cv=4,scoring='neg_mean_squared_error')

In [ ]:
#Como parte de la pregunta tres, calculmao la r cuadrada usando dos folds, y tambein encotrar el promedio, 
Rc=cross_val_score(lre,x_data[['horsepower']], y_data,cv=2)
Rc.mean()

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
yhat = cross_val_predict(lre,x_data[['horsepower']], y_data,cv=4)
yhat[0:5]

In [ ]:
Ejercicio de overfitting, underfitting y seleccion de modelo
#Creamos un objeto que contendra la regesion lineal
lr = LinearRegression()
#Alimentamos el modelo
lr.fit(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']], y_train)

In [ ]:
#Usamos el atributo prediccion con los datos de entrenamiento

yhat_train = lr.predict(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])
yhat_train[0:5]

In [ ]:
#Usamos la prediccion ahora con los datos de prueba
yhat_test = lr.predict(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])
yhat_test[0:5]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
#Analizamos la distribicion de los valores calculados
Title = 'Distribution  Plot of  Predicted Value Using Training Data vs Training Data Distribution'
DistributionPlot(y_train, yhat_train, "Actual Values (Train)", "Predicted Values (Train)", Title)

In [ ]:
itle='Distribution  Plot of  Predicted Value Using Test Data vs Data Distribution of Test Data'
DistributionPlot(y_test,yhat_test,"Actual Values (Test)","Predicted Values (Test)",Title)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
#El overfitting ocurre cuando el modelo se ajusta al ruido, pero no se ajusta al proceso

In [ ]:
#Creamos un set de datos de entrenamiento al 55%
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.45, random_state=0)

In [ ]:
#Usamos la regresion polinomial de la libreria sklearn
pr = PolynomialFeatures(degree=5)
x_train_pr = pr.fit_transform(x_train[['horsepower']])
x_test_pr = pr.fit_transform(x_test[['horsepower']])
pr

In [ ]:
#Ahora usamor la regresion lineal sobre los datos sobre los cuales aplicamos el overfitting
poly = LinearRegression()
poly.fit(x_train_pr, y_train)

In [ ]:
#Usamos el metodo predict para ver la salida de nuestro modelo
yhat = poly.predict(x_test_pr)
yhat[0:5]

In [ ]:
#Imprimimos los valores
print("Predicted values:", yhat[0:4])
print("True values:", y_test[0:4].values)

In [ ]:
"plotemaos nuestros valores
PollyPlot(x_train[['horsepower']], x_test[['horsepower']], y_train, y_test, poly,pr)"

In [ ]:
#Usamos el r cuadrada de los datos de entrenamiento
poly.score(x_train_pr, y_train)

In [ ]:
#Usamos la r cuadrada de los datos de prueba
poly.score(x_test_pr, y_test)

In [ ]:
#Ploteamos la diferencia de los datos
Rsqu_test = []

order = [1, 2, 3, 4]
for n in order:
    pr = PolynomialFeatures(degree=n)
    
    x_train_pr = pr.fit_transform(x_train[['horsepower']])
    
    x_test_pr = pr.fit_transform(x_test[['horsepower']])    
    
    lr.fit(x_train_pr, y_train)
    
    Rsqu_test.append(lr.score(x_test_pr, y_test))

plt.plot(order, Rsqu_test)
plt.xlabel('order')
plt.ylabel('R^2')
plt.title('R^2 Using Test Data')
plt.text(3, 0.75, 'Maximum R^2 ') 

In [ ]:
def f(order, test_data):
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=test_data, random_state=0)
    pr = PolynomialFeatures(degree=order)
    x_train_pr = pr.fit_transform(x_train[['horsepower']])
    x_test_pr = pr.fit_transform(x_test[['horsepower']])
    poly = LinearRegression()
    poly.fit(x_train_pr,y_train)
    PollyPlot(x_train[['horsepower']], x_test[['horsepower']], y_train,y_test, poly, pr)

In [ ]:
interact(f, order=(0, 6, 1), test_data=(0.05, 0.95, 0.05))

In [ ]:
#Creamos un nuevo elemento para la polinomial
pr1=PolynomialFeatures(degree=2)


In [ ]:
#Como parte de la pregunta 4, transformamos los datos de entremaineto y prueba
#con el metodo fit tranform
x_train_pr1=pr1.fit_transform(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])

x_test_pr1=pr1.fit_transform(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])

In [ ]:
#Como parte de la preguna 4d Imprimimos  la forma de nuestros datos
x_train_pr1.shape #there are now 15 features


In [ ]:
#Como parte de la pregunta 4e Aplicamos la regresion lineal a nuestro nuevo frame
poly1=LinearRegression().fit(x_train_pr1,y_train)

In [ ]:
#Como parte de la pregunta e4
#Usamos el metodo predict, par predicir la lsalida d elas caracterisiticas polnomiales
#Usamos la funcion distribution plot para ostrar la distribucion
yhat_test1=poly1.predict(x_test_pr1)

Title='Distribution  Plot of  Predicted Value Using Test Data vs Data Distribution of Test Data'

DistributionPlot(y_test, yhat_test1, "Actual Values (Test)", "Predicted Values (Test)", Title)

In [ ]:
#Usamos la funcion regresion ridge
from sklearn.linear_model import Ridge
#Libreria y nuevo data frame
pr=PolynomialFeatures(degree=2)
x_train_pr=pr.fit_transform(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg','normalized-losses','symboling']])
x_test_pr=pr.fit_transform(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg','normalized-losses','symboling']])

In [ ]:
#definimos elnuevo contendero
RigeModel=Ridge(alpha=1)
#Alimentamos el sistema
RigeModel.fit(x_train_pr, y_train)